In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time
from torchvision.transforms import v2
import random

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [3]:
transform = transforms.Compose([transforms.ToTensor()])
set_seed()

train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform)
validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform)
test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform)

class_names = train_set.classes
num_labels = len(class_names)

In [4]:
from torchvision.models import efficientnet_b0
from torchvision.models import resnext50_32x4d 

In [5]:
def prepare_model_efficient_net(lr):
    model = efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(in_features=1280, out_features=num_labels, bias=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    return model, optimizer, criterion

In [6]:
def prepare_transforms(transform_train, transform_test):
    
    train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform_train)
    validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform_test)
    test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform_test)
        
    data_loader = DataLoader(train_set, batch_size=128, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_val = DataLoader(validate_set, batch_size=128, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(test_set, batch_size=64, num_workers=4, generator=torch.Generator(device='cpu'),persistent_workers=True)

    return data_loader, data_loader_val, data_loader_test





In [7]:
transform_default = transforms.Compose([transforms.ToTensor()])

transform_rotations = transforms.Compose([
    transforms.RandomRotation(degrees = (-30,30)),
    transforms.ToTensor()
])

transform_scaling = transforms.Compose([
    transforms.RandomResizedCrop(size = (32,32),scale=(0.8, 1.0)),  
    transforms.ToTensor()
])

transform_greyscale = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor()
])



transform_random_40 = transforms.Compose([
    transforms.ToTensor(),
        transforms.RandomApply(torch.nn.ModuleList([
     transforms.RandomRotation(degrees = (-30,30)),
 ]), p=0.4),
    transforms.RandomApply(torch.nn.ModuleList([
     transforms.RandomResizedCrop(size = (32,32),scale=(0.8, 1.0)),
 ]), p=0.4)])

transform_random_80 = transforms.Compose([
    transforms.ToTensor(),
        transforms.RandomApply(torch.nn.ModuleList([
     transforms.RandomRotation(degrees = (-30,30)),
 ]), p=0.8),
    transforms.RandomApply(torch.nn.ModuleList([
     transforms.RandomResizedCrop(size = (32,32),scale=(0.8, 1.0)),
 ]), p=0.8)])


transform_scaling_rotating = transforms.Compose([
    transforms.RandomResizedCrop(size = (32,32),scale=(0.8, 1.0)),  
    transforms.RandomRotation(degrees = (-30,30)),
    transforms.ToTensor()
])



#cutmix = v2.CutMix(num_classes=num_labels)



transforms_list = [transform_default,
                  transform_rotations,
                  transform_scaling,
                  transform_greyscale,
                  transform_random_40,
                  transform_random_80,
                  transform_scaling_rotating]

transforms_test = [transform_default,
                   transform_default,
                   transform_default,
                   transform_greyscale,
                   transform_default,
                   transform_default,
                   transform_default]

In [8]:
def model_train(lr,transforms_list, test_list,prepare_model, seed):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    
    for transform, transform_test in zip(transforms_list, test_list):
        data_loader, data_loader_val, data_loader_test = prepare_transforms(transform, transform_test)
        print('transform: '+ str(transform)+ ' test_transform: '+ str(transform_test))
    
        model, optimizer, criterion = prepare_model(lr)
        
        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []
        
        num_epochs = 12
        
        prev_prev_loss = float('inf')
        prev_loss = float('inf')
        curr_loss = float('inf')
        
        for epoch in range(num_epochs): 
            print('Entered the loop')
            model.train()
            total_loss = 0
            correct = 0
            total = 0
            
            for i, (batch_X, batch_Y) in enumerate(data_loader):
                X = batch_X.to(device, non_blocking=True)  
                if(i == 1):
                    print('going')
                Y = batch_Y.to(device, non_blocking=True)  
                optimizer.zero_grad()
                outputs = model(X)
                loss = criterion(outputs, Y)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)
            

            avg_loss = total_loss / len(data_loader)
            train_accuracy = correct / total * 100
            train_losses.append(avg_loss)
            train_accuracies.append(train_accuracy)
            print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
        
            model.eval()
            val_loss = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for batch_X, batch_Y in data_loader_val:
                    X = batch_X.to(device, non_blocking=True)
                    Y = batch_Y.to(device, non_blocking=True)
                    outputs = model(X)
                    loss = criterion(outputs, Y)
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == Y).sum().item()
                    total += Y.size(0)
            avg_val_loss = val_loss / len(data_loader_val)
            val_accuracy = correct / total * 100
            print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")
            
            val_losses.append(avg_val_loss)
            val_accuracies.append(val_accuracy)
    
    
            if(avg_val_loss>curr_loss and curr_loss>prev_loss and prev_loss > prev_prev_loss):
                losses_tr.append(train_losses)
                accuracies_tr.append(train_accuracies) 
                losses_val.append(val_losses)  
                accuracies_val.append(val_accuracies) 
                break
            prev_prev_loss = prev_loss
            prev_loss = curr_loss
            curr_loss = avg_val_loss
        
            if(epoch == num_epochs - 1 ):
                losses_tr.append(train_losses)
                accuracies_tr.append(train_accuracies) 
                losses_val.append(val_losses)  
                accuracies_val.append(val_accuracies) 
    return losses_tr, accuracies_tr, losses_val, accuracies_val




In [11]:
results = []
for i in range(3):
     results.append(model_train(0.001,transforms_list,transforms_test,prepare_model_efficient_net,i))

transform: Compose(
    ToTensor()
) test_transform: Compose(
    ToTensor()
)
Entered the loop
going
Epoch 1, Average Loss: 1.8964
Epoch 1, Validation Loss: 1.7583, Accuracy: 36.44%
Entered the loop
going
Epoch 2, Average Loss: 1.5984
Epoch 2, Validation Loss: 1.4898, Accuracy: 45.37%
Entered the loop
going
Epoch 3, Average Loss: 1.4447
Epoch 3, Validation Loss: 1.4325, Accuracy: 47.85%
Entered the loop
going
Epoch 4, Average Loss: 1.3392
Epoch 4, Validation Loss: 1.3124, Accuracy: 52.66%
Entered the loop
going
Epoch 5, Average Loss: 1.2503
Epoch 5, Validation Loss: 1.4698, Accuracy: 47.10%
Entered the loop
going
Epoch 6, Average Loss: 1.2847
Epoch 6, Validation Loss: 1.2562, Accuracy: 55.05%
Entered the loop
going
Epoch 7, Average Loss: 1.1661
Epoch 7, Validation Loss: 1.1980, Accuracy: 56.72%
Entered the loop
going
Epoch 8, Average Loss: 1.1227
Epoch 8, Validation Loss: 1.1613, Accuracy: 58.01%
Entered the loop
going
Epoch 9, Average Loss: 1.0766
Epoch 9, Validation Loss: 1.1644, Ac

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "D:\anaconda\envs\torch_envi\Lib\site-packages\torch\utils\data\_utils\worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\torch_envi\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\torch_envi\Lib\site-packages\torch\utils\data\_utils\collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\torch_envi\Lib\site-packages\torch\utils\data\_utils\collate.py", line 212, in collate
    collate(samples, collate_fn_map=collate_fn_map)
  File "D:\anaconda\envs\torch_envi\Lib\site-packages\torch\utils\data\_utils\collate.p

In [ ]:
def prepare_model_resnet(lr):
    model = resnext50_32x4d()
    model.fc = nn.Linear(2048, num_labels, bias = True)
    model.conv1 = nn.Conv2d(3,64, kernel_size = (3,3), stride = 1, padding = 1, bias = False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    return model, optimizer, criterion

In [ ]:
results_resnet = []
for i in range(3):
     results.append(model_train(0.001,transforms_list,transforms_test,prepare_model_resnet,i))

In [12]:
results

[([[1.896396836435253,
    1.5984010344201869,
    1.4446623218669132,
    1.3392044288868254,
    1.2502555649896914,
    1.2846737410873175,
    1.1660954192788764,
    1.1227205916392533,
    1.0765761206434532,
    1.0802768372337928,
    1.0993167562410235,
    1.0523471283641728],
   [1.9238745688714765,
    1.6820514527234165,
    1.5477627685124224,
    1.4813589987091043,
    1.3950765215876428,
    1.3410301272842018,
    1.3012588913129135,
    1.2587992758913473,
    1.2426702307367867,
    1.2116063188084147,
    1.2295753918588161,
    1.3419679532843558],
   [1.9199579969387164,
    1.6549083534628153,
    1.5299168178303675,
    1.4252659233117646,
    1.3347575095566837,
    1.2686531329527497,
    1.2098984511738473,
    1.159646605141461,
    1.1191140156747266,
    1.098160932348533,
    1.0716156600551172,
    1.0283333324742587],
   [2.0197226880964907,
    1.740501982075247,
    1.549730440771038,
    1.4311655541373924,
    1.359321067956361,
    1.3115000006827